In [1]:
##importing necessary packages 
import pandas as pd
import numpy as np
import scipy.optimize as opt
import geopy
from geopy.distance import geodesic
from scipy.optimize import differential_evolution

In [2]:
#reading in the csv File
radio_merger = pd.read_csv('radio_merger_data.csv')
radio_merger.head(n=6)

,year,buyer_id,target_id,buyer_lat,buyer_long,target_lat,target_long,price,hhi_target,num_stations_buyer,population_target,corp_owner_buyer
0,2007,1,1,46.592512,-92.549564,44.375073,-92.039543,157763.91,80,3,21676,0
1,2007,2,2,32.578185,-85.349003,33.025375,-86.059702,1472463.20,376,1,11539,0
2,2007,3,3,30.639867,-88.254450,31.122499,-87.766408,3786333.90,129,1,182265,0
3,2007,4,4,38.956806,-94.683236,36.196946,-94.006823,473291.74,188,20,203065,0
4,2007,5,5,41.054082,-73.536216,40.909898,-73.457023,1840579.00,284,0,1493350,0
5,2007,6,6,39.223416,-106.882770,39.269849,-107.023350,1104347.40,51,1,17148,0


In [3]:
radio_merger.describe()

,year,buyer_id,target_id,buyer_lat,buyer_long,target_lat,target_long,price,hhi_target,num_stations_buyer,population_target,corp_owner_buyer
count,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,9.900000e+01,99.000000,99.000000,9.900000e+01,99.000000
mean,2007.545455,25.454545,25.454545,36.909475,-94.392918,37.476019,-94.365015,4.138339e+06,96.353535,18.828283,3.229588e+05,0.020202
std,0.500464,14.709620,14.709620,5.068201,13.957259,4.853933,13.695796,1.468443e+07,82.242463,69.837719,1.098850e+06,0.141407
min,2007.000000,1.000000,1.000000,25.558428,-122.710600,25.369355,-122.433420,5.724072e+04,10.000000,0.000000,2.553000e+03,0.000000
25%,2007.000000,13.000000,13.000000,32.886576,-104.497680,33.687825,-104.876615,4.913584e+05,54.500000,1.000000,2.169800e+04,0.000000
50%,2008.000000,25.000000,25.000000,36.653256,-92.662675,36.900901,-92.122592,1.013110e+06,72.000000,2.000000,4.994800e+04,0.000000
75%,2008.000000,37.500000,37.500000,40.858790,-83.813483,41.145355,-84.212299,2.330153e+06,91.000000,5.500000,1.538550e+05,0.000000
max,2008.000000,54.000000,54.000000,48.704839,-71.411500,48.906401,-70.018443,1.390000e+08,388.000000,591.000000,9.818605e+06,1.000000


In [4]:
##We scale price and population as suggested in the problem. also removed unnecessary variables  
radio_merger['ln_price'] = np.log((radio_merger['price']) / 1000)
radio_merger['ln_popu'] = np.log((radio_merger['population_target']) / 1000)
radio_merger = radio_merger.drop(columns = ['population_target', 'price']) ##not needed
radio_merger

,year,buyer_id,target_id,buyer_lat,buyer_long,target_lat,target_long,hhi_target,num_stations_buyer,corp_owner_buyer,ln_price,ln_popu
0,2007,1,1,46.592512,-92.549564,44.375073,-92.039543,80,3,0,5.061100,3.076206
1,2007,2,2,32.578185,-85.349003,33.025375,-86.059702,376,1,0,7.294692,2.445733
2,2007,3,3,30.639867,-88.254450,31.122499,-87.766408,129,1,0,8.239154,5.205462
3,2007,4,4,38.956806,-94.683236,36.196946,-94.006823,188,20,0,6.159712,5.313526
4,2007,5,5,41.054082,-73.536216,40.909898,-73.457023,284,0,0,7.517835,7.308777
...,...,...,...,...,...,...,...,...,...,...,...,...
94,2008,50,50,38.824690,-104.562030,38.270870,-85.786112,98,0,0,8.019393,6.608130
95,2008,51,51,39.609596,-79.967623,38.820973,-80.042884,34,16,0,6.969570,3.381165
96,2008,52,52,35.294405,-118.905170,37.104785,-107.919850,164,4,0,5.534486,3.938353
97,2008,53,53,40.278498,-76.875207,40.107720,-77.782639,80,2,0,6.787249,5.008085


In [5]:
##adding distance (km)
radio_merger['distance'] = radio_merger.apply(lambda row: (geodesic((row.buyer_lat, row.buyer_long), (row.target_lat, row.target_long)).km), axis = 1)
radio_merger

,year,buyer_id,target_id,buyer_lat,buyer_long,target_lat,target_long,hhi_target,num_stations_buyer,corp_owner_buyer,ln_price,ln_popu,distance
0,2007,1,1,46.592512,-92.549564,44.375073,-92.039543,80,3,0,5.061100,3.076206,249.652058
1,2007,2,2,32.578185,-85.349003,33.025375,-86.059702,376,1,0,7.294692,2.445733,83.008467
2,2007,3,3,30.639867,-88.254450,31.122499,-87.766408,129,1,0,8.239154,5.205462,70.999830
3,2007,4,4,38.956806,-94.683236,36.196946,-94.006823,188,20,0,6.159712,5.313526,312.082957
4,2007,5,5,41.054082,-73.536216,40.909898,-73.457023,284,0,0,7.517835,7.308777,17.343800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2008,50,50,38.824690,-104.562030,38.270870,-85.786112,98,0,0,8.019393,6.608130,1635.068840
95,2008,51,51,39.609596,-79.967623,38.820973,-80.042884,34,16,0,6.969570,3.381165,87.793532
96,2008,52,52,35.294405,-118.905170,37.104785,-107.919850,164,4,0,5.534486,3.938353,1007.567555
97,2008,53,53,40.278498,-76.875207,40.107720,-77.782639,80,2,0,6.787249,5.008085,79.562803


In [6]:
radio_merger.year.describe() ##how many years? we have two years, 2007 and 2008. so we create sub-data for them

count      99.000000
mean     2007.545455
std         0.500464
min      2007.000000
25%      2007.000000
50%      2008.000000
75%      2008.000000
max      2008.000000
Name: year, dtype: float64

In [7]:
#separating the yearly markets  
rd_2007 = radio_merger[radio_merger['year'] == 2007] ##market for 2007
rd_2008 = radio_merger[radio_merger['year'] == 2008] ##market for 2008
years = [rd_2007, rd_2008]

In [8]:
##now separate buyer and target characeristics and creating loops for counterfactuals
buyer_charact = ['year', 'buyer_lat', 'buyer_long', 'buyer_id', 'corp_owner_buyer', 'num_stations_buyer']
target_charact = ['target_lat', 'target_long', 'target_id', 'ln_price', 'hhi_target', 'ln_popu']


cf_rd = [x[buyer_charact].iloc[i].values.tolist() + x[target_charact].iloc[j].values.tolist() \
         for x in years for i in range(len(x)) for j in range(len(x)) if i != j]
cf_rd = pd.DataFrame(cf_rd, columns = buyer_charact + target_charact)
cf_rd

,year,buyer_lat,buyer_long,buyer_id,corp_owner_buyer,num_stations_buyer,target_lat,target_long,target_id,ln_price,hhi_target,ln_popu
0,2007.0,46.592512,-92.549564,1.0,0.0,3.0,33.025375,-86.059702,2.0,7.294692,376.0,2.445733
1,2007.0,46.592512,-92.549564,1.0,0.0,3.0,31.122499,-87.766408,3.0,8.239154,129.0,5.205462
2,2007.0,46.592512,-92.549564,1.0,0.0,3.0,36.196946,-94.006823,4.0,6.159712,188.0,5.313526
3,2007.0,46.592512,-92.549564,1.0,0.0,3.0,40.909898,-73.457023,5.0,7.517835,284.0,7.308777
4,2007.0,46.592512,-92.549564,1.0,0.0,3.0,39.269849,-107.023350,6.0,7.007010,51.0,2.841882
...,...,...,...,...,...,...,...,...,...,...,...,...
4837,2008.0,31.489241,-87.052004,54.0,0.0,1.0,36.900901,-81.494808,49.0,6.227633,127.0,3.472215
4838,2008.0,31.489241,-87.052004,54.0,0.0,1.0,38.270870,-85.786112,50.0,8.019393,98.0,6.608130
4839,2008.0,31.489241,-87.052004,54.0,0.0,1.0,38.820973,-80.042884,51.0,6.969570,34.0,3.381165
4840,2008.0,31.489241,-87.052004,54.0,0.0,1.0,37.104785,-107.919850,52.0,5.534486,164.0,3.938353


In [9]:
##we can see that there is no distance. so we add distance (km) column
cf_rd['distance'] = cf_rd.apply(lambda row: (geodesic((row.buyer_lat, row.buyer_long), (row.target_lat, row.target_long)).km), axis = 1)
cf_rd #now we have distance

,year,buyer_lat,buyer_long,buyer_id,corp_owner_buyer,num_stations_buyer,target_lat,target_long,target_id,ln_price,hhi_target,ln_popu,distance
0,2007.0,46.592512,-92.549564,1.0,0.0,3.0,33.025375,-86.059702,2.0,7.294692,376.0,2.445733,1604.192191
1,2007.0,46.592512,-92.549564,1.0,0.0,3.0,31.122499,-87.766408,3.0,8.239154,129.0,5.205462,1765.973939
2,2007.0,46.592512,-92.549564,1.0,0.0,3.0,36.196946,-94.006823,4.0,6.159712,188.0,5.313526,1160.907565
3,2007.0,46.592512,-92.549564,1.0,0.0,3.0,40.909898,-73.457023,5.0,7.517835,284.0,7.308777,1656.940898
4,2007.0,46.592512,-92.549564,1.0,0.0,3.0,39.269849,-107.023350,6.0,7.007010,51.0,2.841882,1430.892416
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4837,2008.0,31.489241,-87.052004,54.0,0.0,1.0,36.900901,-81.494808,49.0,6.227633,127.0,3.472215,788.794348
4838,2008.0,31.489241,-87.052004,54.0,0.0,1.0,38.270870,-85.786112,50.0,8.019393,98.0,6.608130,761.169624
4839,2008.0,31.489241,-87.052004,54.0,0.0,1.0,38.820973,-80.042884,51.0,6.969570,34.0,3.381165,1033.413180
4840,2008.0,31.489241,-87.052004,54.0,0.0,1.0,37.104785,-107.919850,52.0,5.534486,164.0,3.938353,2014.607173


we start by claculating payoffs without transfers 

In [10]:
##payoff function without transfers
def first_payoff(param_coeffs):
    '''
   We use this function to calculate the payoffs of the mergers, taking note of the inequality

    '''
      
    fmbt_ctt = cf_rd['num_stations_buyer'] * radio_merger['ln_popu'] + param_coeffs[0] * cf_rd['corp_owner_buyer'] * radio_merger['ln_popu'] + param_coeffs[1] * radio_merger['distance']
    fmbtt = radio_merger['num_stations_buyer'] * cf_rd['ln_popu'] + param_coeffs[0] * radio_merger['corp_owner_buyer'] * cf_rd['ln_popu'] + param_coeffs[1] * radio_merger['distance']
    fmbt = radio_merger['num_stations_buyer'] * radio_merger['ln_popu']+ param_coeffs[0] * radio_merger['corp_owner_buyer'] * radio_merger['ln_popu'] + param_coeffs[1] * radio_merger['distance']
    fmbt_ct = cf_rd['num_stations_buyer'] * cf_rd['ln_popu'] + param_coeffs[0] * cf_rd['corp_owner_buyer'] * cf_rd['ln_popu'] + param_coeffs[1] * cf_rd['distance']
     
    k = fmbt_ctt + fmbtt
    j = fmbt + fmbt_ct
    i=(j>k)
    
     
    sumtotal=i.sum()
    return -sumtotal

In [11]:
##we now set bounds and use differential evolution since the maximum score objective function is not smooth and differentiable.
bounds = [(-.5,.5),(-.5,.5)]
results_one = differential_evolution(first_payoff, bounds)
print("Results for model 1:",results_one.x)

Results for model 1: [0.31545952 0.07555443]


Now we calculate payoof with transferss

In [12]:
##payoff with transfers 

def second_payoff(lemma_coeffs):
    
    '''
    This function calculates the payoff functions inside the indication function. If f(b,t) + f(b',t') > + f(b't) + f(b,t')  holds,
    then the indicator equals 1, 0 otherwise.
    
    '''
    
      
    fmbt_ctt_2 = lemma_coeffs[0] * cf_rd['num_stations_buyer'] * radio_merger['ln_popu'] + lemma_coeffs[1] * cf_rd['corp_owner_buyer'] * radio_merger['ln_popu'] + lemma_coeffs[2] * radio_merger['hhi_target'] + lemma_coeffs[3] * radio_merger['distance']
    fmbtt_2 = lemma_coeffs[0] * radio_merger['num_stations_buyer'] * cf_rd['ln_popu'] + lemma_coeffs[1] * radio_merger['corp_owner_buyer'] * cf_rd['ln_popu'] + lemma_coeffs[2] * radio_merger['hhi_target'] + lemma_coeffs[3] * radio_merger['distance']
    fmbt_2 = lemma_coeffs[0] * radio_merger['num_stations_buyer'] * radio_merger['ln_popu'] + lemma_coeffs[1] * radio_merger['corp_owner_buyer'] * radio_merger['ln_popu'] + lemma_coeffs[2] * radio_merger['hhi_target'] + lemma_coeffs[3] * radio_merger['distance']
    fmbt_ct_2 = lemma_coeffs[0] * cf_rd['num_stations_buyer'] * cf_rd['ln_popu'] + lemma_coeffs[1] * cf_rd['corp_owner_buyer'] * cf_rd['ln_popu'] + lemma_coeffs[2]*cf_rd['hhi_target'] + lemma_coeffs[3] * cf_rd['distance']
 
    
    k = fmbt_ctt_2 + fmbtt_2
    j = fmbt_2 + fmbt_ct_2
    i=(j>k)
    
    
    
    
    # Summing the total and returning -total since we maximize our objective func by using a minimizer method 
    totalsum_2=i.sum()
    return -totalsum_2

In [13]:
bounds_2 = [(-.5,.5),(-.5,.5),(-.5,.5),(-.5,.5)]
results_two = differential_evolution(second_payoff, bounds_2)
print("Results for model 2:", results_two.x)

Results for model 2: [ 0.36152881  0.28230951 -0.01462136  0.02653222]
